<a href="https://colab.research.google.com/github/girons97/CBE20255/blob/master/Copy_of_vessel_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import datetime
import matplotlib.dates as mdates
import csv
from collections import defaultdict
header = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' }
vessel_names_file = 'vessel_names.csv'
container_names_file = 'containers.csv'

from google.colab import files
uploaded = files.upload()

In [ ]:
def file_to_list(csv_file):
    with open(csv_file, encoding='utf-8-sig') as f:
        return list(csv.reader(f))

vessels = file_to_list(vessel_names_file)

In [ ]:
def containers_to_dict(csv_file):
    containers = file_to_list(csv_file)
    containers_dict = defaultdict(list)
    for container in containers:
        containers_dict[container[0]].append(container[1:])
    return containers_dict


[['L4638', 'MRSU4425782', '13/04/2022', '06:30:00'],
 ['L4639', 'MRSU4425782', '14/04/2022', '07:30:00'],
 ['L4640', 'MRSU4425782', '15/04/2022', '08:30:00'],
 ['L4641', 'MRSU4425782', '15/04/2022', '09:30:00'],
 ['L4642', 'MRSU4425782', '16/04/2022', '10:30:00']]

In [ ]:
def maersk_tracker(container):
    
    url = "https://api.maersk.com/track/{}?operator=MAEU".format(container)
    headers = {
        "accept": "application/json",
        "accept-encoding": "gzip, deflate, br",
        "user-agent": "Mozilla/5.0",
        "x-requested-with": "XMLHttpRequest"
    }
    response = requests.get(url, headers=headers)

    return response.json()

maersk_tracker("MRSU4425782")

{'isContainerSearch': True,
 'origin': {'terminal': 'Beilun Container Terminal Phase 4',
  'geo_site': '33R7MNJ1ZSB4I',
  'city': 'Ningbo',
  'state': 'Zhejiang',
  'country': 'China',
  'country_code': 'CN',
  'geoid_city': '104T898SJZ6GU',
  'site_type': 'TERMINAL'},
 'destination': {'terminal': 'FELIXSTOWE TRINITY TERMINAL',
  'geo_site': '0MZSJ9SROEGKD',
  'city': 'Felixstowe',
  'state': '',
  'country': 'United Kingdom',
  'country_code': 'GB',
  'geoid_city': '23KBBVVUYELJT',
  'site_type': 'TERMINAL'},
 'containers': [{'container_num': 'MRSU4425782',
   'container_size': '40',
   'container_type': 'Dry',
   'iso_code': '42G0',
   'operator': 'MAEU',
   'locations': [{'terminal': 'Beilun Container Terminal Phase 4',
     'geo_site': '33R7MNJ1ZSB4I',
     'city': 'Ningbo',
     'state': 'Zhejiang',
     'country': 'China',
     'country_code': 'CN',
     'geoid_city': '104T898SJZ6GU',
     'site_type': 'TERMINAL',
     'events': [{'activity': 'GATE-IN',
       'stempty': False,
 

In [ ]:
def get_vessel_ids(vessels):
    '''Input .csv file with three columns. Col 1: Vessel code, Col 2: Vessel name, Col 3: IMO number
    Output list of Col 1: Vessel code, Col 2: Vessel name, Col 3: IMO number, Col 4: MMSI number, Col 5: Marine traffic ShipID
    This is done via extracting from the redirected link e.g./ais/details/ships/shipid:4975268/mmsi:538007378/imo:9769283/vessel:MOL_TRUST'''
    headers = {
            "accept": "application/json",
            "accept-encoding": "gzip, deflate",
            "user-agent": "Mozilla/5.0",
            "x-requested-with": "XMLHttpRequest"
        }
    for vessel in vessels: #for each line in the input csv file 
        marine_traffic_req  = requests.get('https://www.marinetraffic.com/en/ais/details/ships/imo:'+vessel[2], headers=header, allow_redirects=False)
        vessel.append(str(marine_traffic_req.headers['Location'].split('mmsi:')[1].split('/')[0])) #Get the vessel MMSI
        vessel.append(str(marine_traffic_req.headers['Location'].split('shipid:')[1].split('/')[0])) #Get the SHIP ID

    return vessels
vessels = get_vessel_ids(vessels)

In [ ]:
def get_ship_position(vessels):

    for vessel in vessels:

        url = "https://www.marinetraffic.com/vesselDetails/latestPosition/shipid:{}".format(vessel[4])

        headers = {
            "accept": "application/json",
            "accept-encoding": "gzip, deflate",
            "user-agent": "Mozilla/5.0",
            "x-requested-with": "XMLHttpRequest"
        }

        response = requests.get(url, headers=headers).json()
        

        # def cleanNullTerms(response):
        #     clean_response = {}
        #     for k, v in response.items():
        #         if isinstance(v, dict):
        #             nested = cleanNullTerms(v)
        #             if len(nested.keys()) > 0:
        #                 clean_response[k] = nested
        #         elif v is not None:
        #             clean_response[k] = v
        #         else:
        #             clean_response[k] = "None"
        #     return clean_response
            
        # response = cleanNullTerms(response)

        lp = datetime.datetime.utcfromtimestamp(response["lastPos"])
        vessel.append(lp)
        vessel.append(response["lat"])
        vessel.append(response["lon"])
        vessel.append(response["shipStatus"])

        try:
            etd = datetime.datetime.utcfromtimestamp(response["departurePort"]["timestamp"])
            vessel.append(response["departurePort"]["name"])
            vessel.append(etd)
        except:
            vessel.append("None")
            vessel.append("None")
        
        try:
            eta = datetime.datetime.utcfromtimestamp(response["arrivalPort"]["timestamp"])
            vessel.append(response["arrivalPort"]["name"])
            vessel.append(eta)
        except:
            vessel.append("None")
            vessel.append("None")
        

        if response["currentPortName"] is None:
            vessel.append("None")
        else:
            vessel.append(response["currentPortName"])

    return vessels
vessels = get_ship_position(vessels)
vessels


[['MAE2031',
  'Maersk Euphrates',
  '9694567',
  '538005747',
  '714735',
  datetime.datetime(2022, 3, 13, 22, 43, 21),
  37.59353,
  6.21659,
  'Underway using Engine',
  'THAMES',
  datetime.datetime(2022, 3, 7, 22, 48),
  'PORT SAID',
  datetime.datetime(2022, 3, 17, 16, 0),
  'None'],
 ['MSC5063',
  'Bg Onyx',
  '9436197',
  '255772000',
  '364673',
  datetime.datetime(2022, 3, 13, 22, 43, 17),
  52.00881,
  3.829252,
  'Underway using Engine',
  'GRANGEMOUTH',
  datetime.datetime(2022, 3, 12, 21, 55),
  'ROTTERDAM',
  datetime.datetime(2022, 3, 13, 22, 0),
  'None'],
 ['CMA4052',
  'Msc Geneva',
  '9320427',
  '636093051',
  '300155',
  datetime.datetime(2022, 3, 13, 21, 0, 20),
  44.54929,
  -6.98833,
  'Underway using Engine',
  'ANTWERP',
  datetime.datetime(2022, 3, 11, 10, 23),
  'SINES',
  datetime.datetime(2022, 3, 15, 12, 0),
  'None'],
 ['ONE3042',
  'Suecia Seaways',
  '9153020',
  '220284000',
  '158822',
  datetime.datetime(2022, 3, 13, 22, 42, 14),
  51.90767,
  4.36

In [ ]:
def berth_slot(vessels):
    
    headers = {
        "accept": "application/json",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Mozilla/5.0",
        "x-requested-with": "XMLHttpRequest"
    }
    
    pfl_url = "https://www.portoffelixstowe.co.uk/sailing-schedule/shipping-information/#Arrivals"
    felixstowe_port_html = requests.get(pfl_url, headers=headers, verify=False)
    felixstowe_movements = pd.read_html(felixstowe_port_html.text)
    felixstowe_arrivals = felixstowe_movements[0]
    felixstowe_arrivals["Vessel Name"] = felixstowe_arrivals["Vessel Name"].str.upper()

    pso_url = r"https://www.southamptonvts.co.uk/BackgroundSite/Ajax/LoadXmlFileWithTransform?xmlFilePath=D:\ftp\southampton\Shipping_Movements\sot_extract.xml&xslFilePath=D:\wwwroot\CMS_Southampton\content\files\assets\sotextract.xsl&w=8"
    southampton_port_html= requests.get(pso_url, headers=headers)
    southampton_movements = pd.read_html(southampton_port_html.text)
    southampton_arrivals = southampton_movements[0]

    for vessel in vessels:
        if vessel[1].upper() in felixstowe_arrivals["Vessel Name"].values:
            vessel.append("FELIXSTOWE")
            vessel.append(felixstowe_arrivals.loc[felixstowe_arrivals['Vessel Name'] == vessel[1].upper()]["ArrivalDate and Time"].max())

        elif vessel[1].upper() in southampton_arrivals["Ship Name"].values:
            vessel.append("SOUTHAMPTON")
            vessel.append(southampton_arrivals.loc[southampton_arrivals['Ship Name'] == vessel[1].upper()]["Estimated Time Of Arrival"].max())

        else:
            vessel.append("None")
            vessel.append("N/A")
    
    return vessels


vessels = berth_slot(vessels) 
vessels

C:\Users\georg\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portoffelixstowe.co.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[['MAE2031',
  'Maersk Euphrates',
  '9694567',
  '538005747',
  '714735',
  datetime.datetime(2022, 3, 13, 22, 43, 21),
  37.59353,
  6.21659,
  'Underway using Engine',
  'THAMES',
  datetime.datetime(2022, 3, 7, 22, 48),
  'PORT SAID',
  datetime.datetime(2022, 3, 17, 16, 0),
  'None',
  'None',
  'N/A'],
 ['MSC5063',
  'Bg Onyx',
  '9436197',
  '255772000',
  '364673',
  datetime.datetime(2022, 3, 13, 22, 43, 17),
  52.00881,
  3.829252,
  'Underway using Engine',
  'GRANGEMOUTH',
  datetime.datetime(2022, 3, 12, 21, 55),
  'ROTTERDAM',
  datetime.datetime(2022, 3, 13, 22, 0),
  'None',
  'FELIXSTOWE',
  '19/03/22 16:30'],
 ['CMA4052',
  'Msc Geneva',
  '9320427',
  '636093051',
  '300155',
  datetime.datetime(2022, 3, 13, 21, 0, 20),
  44.54929,
  -6.98833,
  'Underway using Engine',
  'ANTWERP',
  datetime.datetime(2022, 3, 11, 10, 23),
  'SINES',
  datetime.datetime(2022, 3, 15, 12, 0),
  'None',
  'None',
  'N/A'],
 ['ONE3042',
  'Suecia Seaways',
  '9153020',
  '220284000',
  

In [ ]:
vessel_headers = ["Vessel code", "Vessel name", "IMO", "MMSI", "Ship ID", "Last reported", "Lattitude", "Longtitude", "Vessel status",
"Current location", "ETD", "Last port", "ETA", "Next port", "Berthing location", "Berthing slot"]
vessels_df = pd.DataFrame(vessels, columns=vessel_headers)


In [ ]:
vessels_df.to_csv("vessels_df.csv")

In [ ]:
def one_line_tracker(container):

    headers = {
        "accept": "application/json",
        "accept-encoding": "gzip, deflate, br",
        "user-agent": "Mozilla/5.0",
        "x-requested-with": "XMLHttpRequest"
    }

    url_1 = "https://ecomm.one-line.com/ecom/CUP_HOM_3301GS.do?_search=false&rows=10000&page=1&sidx=&sord=asc&f_cmd=121&search_type=A&search_name={}&cust_cd=".format(container)
    
    response_1 = requests.get(url_1, headers=headers)
    cop_no = response_1.json()["list"][0]["copNo"]
    
    url_2 = "https://ecomm.one-line.com/ecom/CUP_HOM_3301GS.do?f_cmd=125&cntr_no={}&bkg_no=&cop_no={}".format(container, cop_no)
    response_2 = requests.get(url_2, headers=headers)
    
    return response_2.json()

one_line_tracker("ONEU0006598")


In [ ]:
def msc_tracker(container):
    





msc_tracker("MEDU6997295")

'CVLC2201061112'